In [9]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src')
import xgboost as xgb
import pandas as pd
from features.UserJoin import UserJoin
from features.UserJoin import submit, diff, plt_month, plt_day, load_ids, diff_set, infot
import config as C
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
from sklearn.metrics import f1_score
from pathlib import Path
from functools import reduce
import random
from pathlib import Path

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
# 2个挖矿
ids_4_2 = {2745781539, 2602819207, 1429892107, 2624797677, }
# 1个挖矿
ids_3_1 = {1429892107, 2602819207, 2624797677}
# 2个挖矿
ids_3_2 = {2745781539, 2602819207, 1912367373}

ids_2_1 = {2745781539, 2602819207}

ids_7_2 = {2745781539, 2602819207, 179569085, 2624797677, 2194274407, 2044965214, 2115712219}

todo = [
    (ids_4_2, 2),
    (ids_3_1, 1),
    (ids_3_2, 2),
    (ids_2_1, 1),
    (ids_7_2, 2)
]


def check(known_true_ids=set(), known_false_ids=set()):
    def pre(ids, num_1):
        t = ids - known_true_ids
        t = t - known_false_ids
        num_1 -= len(ids & known_true_ids)
        return t, num_1

    for i, num_1i in todo:
        i, num_1i = pre(i, num_1i)
        for j, num_1j in todo:
            j, num_1j = pre(j, num_1j)

            t = i.copy()
            num_1 = num_1i
            if len(i & j) == len(j):
                t -= j
                num_1 -= num_1j

            if len(t) == 0:
                continue
            if len(t) == (len(t) - num_1):
                print(f'{t} 里有 {len(t) - num_1} 个 0')
            if len(t) == num_1:
                print(f'{t} 里有 {num_1} 个 1')

check()

{2745781539} 里有 1 个 1
{1912367373} 里有 1 个 1


In [48]:
check(known_true_ids={1912367373, 2745781539})

{2602819207} 里有 1 个 0
{2602819207} 里有 1 个 0
{2602819207} 里有 1 个 0
{2602819207} 里有 1 个 0
{2602819207} 里有 1 个 0
{2602819207} 里有 1 个 0


In [53]:
check(known_true_ids={1912367373, 2745781539}, known_false_ids={2602819207})

In [55]:
for ids, num1 in todo:
    print(ids - C.true_ids - C.false_ids, len(ids - C.true_ids - C.false_ids), num1 - len(ids & C.true_ids))


set() 0 0
set() 0 0
set() 0 0
set() 0 0
set() 0 0


In [7]:
true = set(random.sample(C.true_ids, int(len(C.true_ids)/1.6)))
false = set(random.sample(C.false_ids, int(len(C.false_ids)/3.3)))
print(len(true), len(false))
# 新增2个挖矿
t = true | false | C.tids_2
len(t)

18 3


23

In [8]:
submit(t, 'submit_3_15_1.csv', True)

23


,id,label
0,179406029,0
1,179406030,0
2,179406094,0
3,179406097,0
4,179406099,0
...,...,...
15374,2849971818,0
15375,2850017469,0
15376,2850017472,0
15377,2851323065,0


In [56]:
print('true ids', len(C.true_ids), 'false ids', len(C.false_ids))

true ids 32 false ids 19


In [58]:
for i in Path('submit_csv').glob('3_10_base_*.csv'):
    ids = load_ids(i.name)
    num_1 = int(i.name.split('_')[-1].replace('.csv', ''))

    num_1 -= len(ids & C.true_ids)
    ids = ids - C.true_ids - C.false_ids

    print(i.name, len(ids), num_1)

3_10_base_98_33.csv 57 3
3_10_base_32_27.csv 0 0
3_10_base_175_34.csv 133 3
3_10_base_544_35.csv 502 4
3_10_base_63_29.csv 27 1
3_10_base_34_27.csv 0 0
3_10_base_35_27.csv 0 0
3_10_base_32_24.csv 0 0
